In [ ]:
# DPO plotted

: 

In [12]:
import numpy as np

# Define the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Define the DPO loss components
def dpo_loss(pi_theta_w, pi_theta_l, pi_ref_w, pi_ref_l, beta=1.0):
    """
    Compute the DPO loss given probabilities and scaling factor beta.
    
    Parameters:
    - pi_theta_w: P(y_w | x) from the current model (float or array-like)
    - pi_theta_l: P(y_l | x) from the current model (float or array-like)
    - pi_ref_w: P(y_w | x) from the reference model (float or array-like)
    - pi_ref_l: P(y_l | x) from the reference model (float or array-like)
    - beta: Scaling factor (default = 1.0)
    
    Returns:
    - Loss value (float or array-like)
    """
    # Term (3): Reward for preferred output
    reward_w = beta * np.log(pi_theta_w / pi_ref_w)
    
    # Term (4): Reward for dispreferred output
    reward_l = beta * np.log(pi_theta_l / pi_ref_l)
    
    # Term (2): Sigmoid of the reward difference
    sigmoid_term = sigmoid(reward_w - reward_l)
    
    # Term (1): Negative log of the sigmoid term
    loss = -np.log(sigmoid_term)
    
    return loss

# Example input values
pi_theta_w = np.linspace(0.001, 0.01, 100)  # P(y_w | x) from the current model
pi_theta_l = .01 - pi_theta_w                # P(y_l | x) from the current model
pi_ref_w = 0.006                             # P(y_w | x) from the reference model (constant for simplicity)
pi_ref_l = 0.004                             # P(y_l | x) from the reference model (constant for simplicity)
beta = 1.0                                 # Scaling factor

# Compute the DPO loss
loss_values = dpo_loss(pi_theta_w, pi_theta_l, pi_ref_w, pi_ref_l, beta=beta)

# Plotting the loss
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(pi_theta_w, loss_values, label='DPO Loss', color='blue')
plt.title("DPO Loss as $P(y_w)$ Increases", fontsize=14)
plt.xlabel("$P(y_w)$ (Current Model)", fontsize=12)
plt.ylabel("DPO Loss", fontsize=12)
plt.grid(alpha=0.5)
plt.legend(fontsize=12)
plt.show()


/var/folders/b_/l_rcmwq56dq2902h_83wm9t80000gn/T/ipykernel_82794/3899664489.py:26: RuntimeWarning: divide by zero encountered in log
  reward_l = beta * np.log(pi_theta_l / pi_ref_l)


In [8]:
import numpy as np 

logits = np.linspace(-3, +3, 11)
# run logistic function on the logits
probs = 1 / (1 + np.exp(-logits))
odds = np.exp(logits)

# print out the values in a nice table
print("logit\t| sigmoid(x) [prob]\t| exp(x) [odds]")
print("-" * 30)
for i in range(len(logits)):
    print(f"{logits[i]:.4f}\t| {probs[i]:.4f}\t\t| {odds[i]:.4f}")


logit	| sigmoid(x) [prob]	| exp(x) [odds]
------------------------------
-3.0000	| 0.0474		| 0.0498
-2.4000	| 0.0832		| 0.0907
-1.8000	| 0.1419		| 0.1653
-1.2000	| 0.2315		| 0.3012
-0.6000	| 0.3543		| 0.5488
0.0000	| 0.5000		| 1.0000
0.6000	| 0.6457		| 1.8221
1.2000	| 0.7685		| 3.3201
1.8000	| 0.8581		| 6.0496
2.4000	| 0.9168		| 11.0232
3.0000	| 0.9526		| 20.0855


In [20]:
import numpy as np
import matplotlib.pyplot as plt

# Define the loss function (binary cross-entropy) and its gradient
def binary_cross_entropy_loss(logit):
    # Convert logit to probability using sigmoid
    prob = 1 / (1 + np.exp(-logit))
    # Binary cross-entropy loss for the positive class (target = 1)
    return -np.log(prob)

def gradient_of_loss(logit):
    # Gradient of binary cross-entropy w.r.t. the logit
    prob = 1 / (1 + np.exp(-logit))
    return prob - 1  # derivative of BCE loss when target = 1

# Generate logits from 0 to 5.0
logits = np.linspace(0, 5, 100)

# Compute loss and gradient
losses = binary_cross_entropy_loss(logits)
gradients = gradient_of_loss(logits)



# Define a specific logit value to annotate
annotate_logit = 0
annotate_gradient = gradient_of_loss(annotate_logit)

# Plot the loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(logits, losses, label="Binary Cross-Entropy Loss")
plt.title("Binary Cross-Entropy Loss as a Function of Logit")
plt.xlabel("Difference of language model and reference model logit")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

# Plot the gradient
plt.subplot(1, 2, 2)
plt.plot(logits, gradients, label="Gradient of Loss", color="orange")
plt.scatter([annotate_logit], [annotate_gradient], color="red", label=f"Point at logit={annotate_logit}")
plt.annotate(f"Gradient = {annotate_gradient:.2f}",
             (annotate_logit, annotate_gradient),
             textcoords="offset points", xytext=(50, 10), ha="center", color="red")
plt.title("Gradient of Loss as a Function of Logit")
plt.xlabel("Difference of language model and reference model logit")
plt.ylabel("Gradient")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [15]:
import numpy as np
import matplotlib.pyplot as plt

# Define the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the binary cross-entropy loss function
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-15  # To avoid log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Define the gradient of the binary cross-entropy loss with respect to y_pred
def bce_gradient(y_true, y_pred):
    epsilon = 1e-15  # To avoid division by zero
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return (y_pred - y_true) / (y_pred * (1 - y_pred))

# Generate a range of logit values
logits = np.linspace(-10, 10, 400)
# Compute the corresponding sigmoid outputs (predicted probabilities)
pred_probs = sigmoid(logits)

# Set the true label
y_true = 0.6

# Compute the loss and gradient for the given true label
loss = binary_cross_entropy(y_true, pred_probs)
gradient = bce_gradient(y_true, pred_probs)

# Plot the loss function and its gradient
plt.figure(figsize=(10, 6))
plt.plot(pred_probs, loss, label='Binary Cross-Entropy Loss', color='blue')
plt.plot(pred_probs, gradient, label='Gradient of Loss', color='red', linestyle='--')
plt.xlabel('Predicted Probability')
plt.ylabel('Value')
plt.title('Binary Cross-Entropy Loss and Its Gradient vs. Predicted Probability, y_true=0.6')
plt.legend()
plt.grid(True)

# Set y-axis limits to [-1, 1]
plt.ylim(-3, 3)

# Annotate the minimum loss point
min_index = np.argmin(loss)
min_pred_prob = pred_probs[min_index]
min_loss = loss[min_index]
plt.scatter(min_pred_prob, min_loss, color='black')
plt.annotate(f'Min Loss at prob \n({min_pred_prob:.2f})',
             xy=(min_pred_prob, min_loss),
             xytext=(min_pred_prob + 0.1, min_loss+0.2),
             arrowprops=dict(facecolor='black', arrowstyle='->'),
             horizontalalignment='left')

plt.show()


In [6]:
def softmax(logits):
    exp_values = np.exp(logits - np.max(logits, axis=-1, keepdims=True))  # Numerical stability trick
    return exp_values / np.sum(exp_values, axis=-1, keepdims=True)

# Generate logits from -3 to 3
logits = np.linspace(-3, 3, 100)

# Compute softmax probability for the first entry of a 2-element vector [z, 0]
softmax_values = softmax(np.column_stack((logits, np.zeros_like(logits))))[:, 0]


# Define initial point and gradient update
z_t0 = 0  # Initial value of z
grad_update = 0.2  # Gradient update step
z_t1 = z_t0 + grad_update  # Updated z value
softmax_z_t1 = softmax(np.array([[z_t1, 0]]))[0, 0]  # Compute new softmax probability


# Define new update steps
update_step = 0.2  # Single update step
z_t1 = z_t0 - update_step  # Move in the direction of increasing logit

# Compute the softmax probability for the updated z
softmax_z_t1 = softmax(np.array([[z_t1, 0]]))[0, 0]

# Define a second update step (10x update)
update_step_x10 = 2.0  # Large update step
z_t2 = z_t0 - update_step_x10  # Optimizing for a bigger logit
softmax_z_t2 = softmax(np.array([[z_t2, 0]]))[0, 0]


# Plot the softmax function
plt.figure(figsize=(6, 4))
plt.plot(logits, softmax_values, label="Softmax Function (P(y=1 | [z, 0]))", color="blue")
plt.xlabel("Logits (z)")
plt.ylabel("Softmax Output")
plt.title("Softmax Probability for First Class")
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.6)  # Reference line at 0.5
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.6)  # Reference line at 0
plt.ylim(0, 1)  # Set y-axis scale from 0 to 1
plt.legend()
plt.grid(True)

# First underbrace (red) from 0.0 to 0.2
plt.annotate("", xy=(0.2, 0.2), xytext=(0.0, 0.2),
             arrowprops=dict(arrowstyle="|-|", color="red", lw=1.5))
plt.text(0.1, 0.15, "Update: 0.2", ha="center", fontsize=12, color="red")

# Second underbrace (green) from 0.0 to 2.0, placed at the bottom
plt.annotate("", xy=(2.0, 0.05), xytext=(0.0, 0.05),
             arrowprops=dict(arrowstyle="|-|", color="green", lw=1.5))
plt.text(1.0, 0.01, "Update x10: 2.0", ha="center", fontsize=12, color="green")

# Change markers to a different symbol (e.g., triangles)
plt.scatter([0.2], [0.55], color="red", marker='^', s=100, zorder=3, label="Updated z_t (0.2, 0.55)")
plt.text(0.2, 0.55 + 0.03, "(0.2, 0.55)", ha="center", fontsize=10, color="red")

plt.scatter([2.0], [0.88], color="green", marker='v', s=100, zorder=3, label="Updated z_t x10 (2.0, 0.88)")
plt.text(2.0, 0.88 + 0.03, "(2.0, 0.88)", ha="center", fontsize=10, color="green")

# Show the plot
plt.show()
